In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/TwitterFriends/data.csv


# Exploring and cleaning the data

In [2]:
# There are problems reading the CSV file since there are commas in the array of the last columns "friends"
# We will use a regular expression to consider only the commas that are NOT followed by a whitespace
twitterData = pd.read_csv("/kaggle/input/TwitterFriends/data.csv", sep=',(?=\S)', engine='python')

#Take a look at the data
twitterData.head()

,id,screenName,tags,avatar,followersCount,friendsCount,lang,lastSeen,tweetId,friends
0,"""1969527638""","""LlngoMakeEmCum_""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/534286217...",319,112,"""en""",1472271687519,"""769310701580083200""","[ ""1969574754"", ""1969295556"", ""1969284056"", ""1..."
1,"""51878493""","""_notmichelle""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/761977602...",275,115,"""en""",1472270622663,"""769309490038439936""","[ ""60789485"", ""2420931980"", ""2899776756"", ""127..."
2,"""1393409100""","""jesseayye""","[ ""#narcos"" ]","""http://pbs.twimg.com/profile_images/713282938...",120,107,"""en""",1472804144409,"""771622623583186944""","[ ""86868062"", ""19697415"", ""2998836604"", ""45629..."
3,"""232891415""","""MrBrianLloyd""","[ ""#gloryoutnow"" ]","""http://pbs.twimg.com/profile_images/133440668...",492,325,"""en""",1472269186776,"""769308131696467969""","[ ""361335082"", ""1405248468"", ""24626354"", ""7256..."
4,"""710130422907207680""","""sarahdorat_16""","[ ""#nationaldogday"" ]","""http://pbs.twimg.com/profile_images/767180520...",128,218,"""en""",1472271397356,"""769309769144283137""","[ ""1571896093"", ""768938323612008448"", ""2548665..."


In [7]:
twitterData.info()
twitterData = twitterData[['id', 'screenName', 'friends']]
twitterData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          40000 non-null  object
 1   screenName  40000 non-null  object
 2   friends     40000 non-null  object
dtypes: object(3)
memory usage: 937.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          40000 non-null  object
 1   screenName  40000 non-null  object
 2   friends     40000 non-null  object
dtypes: object(3)
memory usage: 937.6+ KB


Many columns do not have the correct data type; we will transform them to correct data  
We will remove quotes and turn dtype to integer if needed. Arrays will not be modified for now«

In [49]:
first_100_rows = twitterData.iloc[:1000]
first_100_rows.info()
first_100_rows.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   object
 1   screenName  1000 non-null   object
 2   friends     1000 non-null   object
dtypes: object(3)
memory usage: 23.6+ KB


,id,screenName,friends
0,"""1969527638""","""LlngoMakeEmCum_""","[ ""1969574754"", ""1969295556"", ""1969284056"", ""1..."
1,"""51878493""","""_notmichelle""","[ ""60789485"", ""2420931980"", ""2899776756"", ""127..."
2,"""1393409100""","""jesseayye""","[ ""86868062"", ""19697415"", ""2998836604"", ""45629..."
3,"""232891415""","""MrBrianLloyd""","[ ""361335082"", ""1405248468"", ""24626354"", ""7256..."
4,"""710130422907207680""","""sarahdorat_16""","[ ""1571896093"", ""768938323612008448"", ""2548665..."


In [ ]:
twitterData.head()

# 1- What is the structure of the graph ?

In [59]:
# Create an empty graph
import networkx as nx
import ast 
import re
G = nx.Graph()
all_user_ids = set()
for index, row in first_100_rows.iterrows():
    user_id = row['id'].replace('"', '') 
    all_user_ids.add(int(user_id))
print(all_user_ids)
# Iterate through the DataFrame rows
for index, row in first_100_rows.iterrows():
    user_id = row['id']
    friends_list = re.findall(r'"(.*?)"', str(row['friends']))
    #friends_list = ast.literal_eval(row['friends'])  # Convert string representation of list to a Python list
    
    # Add edges between the user and their friends
    for friend_id in friends_list:
        mod_friend = friend_id.replace('"', '') 
        if int(mod_friend) in all_user_ids:
            G.add_edge(user_id.replace('"', ''), mod_friend.replace('"', ''))
            


{710130422907207680, 708420695508783104, 736866783794221057, 738833141905903616, 716832511821615104, 731223079955795968, 765096881949028352, 729086055274582016, 701740027995361280, 717045547559878656, 3518961674, 732645396418297858, 1122222090, 56836105, 553740298, 15181839, 1558700048, 1047771152, 52058127, 134123540, 92473365, 3332075541, 98949148, 2898726941, 3367389215, 165318690, 398585891, 2295584804, 159270947, 858712106, 69611563, 2841047082, 2416820269, 936497202, 2344163382, 295616567, 280127542, 3002349627, 3409035328, 186519621, 19515463, 1149831241, 2313809995, 134727759, 359163989, 2693212248, 3163957336, 2800101466, 128950362, 31236188, 2939168865, 238979169, 263043173, 2549469287, 531894376, 834988136, 269768813, 30214256, 137588849, 471703668, 4890830966, 2243209334, 377319544, 580505722, 56899709, 2441775230, 321376383, 709037473209454592, 470128769, 762640656250638336, 760986390691778560, 26347653, 131649669, 2378113158, 4827611272, 755555729923117057, 2760577158, 28

According to info(), there are 40,000 entries. However, we can make sure all the entries are unique ids. 

In [71]:
import matplotlib.pyplot as plt
plt.figure(figsize=(100,100))
pos = nx.spring_layout(G)  # Define node positions using a layout algorithm (spring_layout in this case)
nx.draw(G, pos, with_labels=True, node_size=20000, node_color='skyblue', font_weight='bold', edge_color='red', width=15)


(Also, does this question imply we also want to count the users in the friend lists? If necessary, count unique ids from all friends.)

# 2- What are the cliques that can be formed here ?

In [65]:
maximal_cliques = list(nx.find_cliques(G))

# Print the maximal cliques
print("Maximal Cliques:")
for clique in maximal_cliques:
    print(clique)

Maximal Cliques:
['226018414', '24272045']
['1968636487', '1971014755']
['4649254875', '2877586418']
['4649254875', '1295650423']
['2356581390', '4656527605']
['711121566', '796932380']
['1971245030', '1971261620']
['1971245030', '1970490685']
['434081382', '789526806']
['1974555133', '1975276057']
['2220768493', '707053416258797568']
['1295650423', '219050834']
['1969604874', '1969547112']
['219050834', '2741348007']
['6324852', '16353989']
['6324852', '123666386']
['36654652', '25562113']
['3410763455', '761541223601491968', '759840044882489344']
['3410763455', '702362283859648513']
['3410763455', '816123902']
['3410763455', '707642488186658816']
['3410763455', '4243203073']
['3410763455', '2474916424']
['3410763455', '3603567192']
['3410763455', '505781191']
['3410763455', '2841047082']
['3410763455', '789526806']
['3410763455', '4920627202']
['3410763455', '461010831']
['2916011432', '726951769']
['720954450', '422646582']
['3246926345', '3993741502']
['725931738', '2800101466']
['

# 3- How many components are there ?

In [78]:
components = list(nx.connected_components(G))

# Identify the largest component
largest_component = max(components, key=len)

# Visualization of entire graph with the largest component highlighted
plt.figure(figsize=(200, 200))

pos = nx.spring_layout(G)
nx.draw(
    G,
    pos,
    with_labels=True,
    node_size=20000,
    node_color='skyblue',
    font_weight='bold',
    edge_color='black',
    width=5.0
)

# Draw a boundary around the largest component
node_positions = {node: pos[node] for node in largest_component}
node_values = list(node_positions.values())
min_x, max_x = min(node_values, key=lambda x: x[0])[0], max(node_values, key=lambda x: x[0])[0]
min_y, max_y = min(node_values, key=lambda x: x[1])[1], max(node_values, key=lambda x: x[1])[1]
plt.gca().add_patch(plt.Rectangle((min_x - 0.1, min_y - 0.1), max_x - min_x + 0.2, max_y - min_y + 0.2, fill=False, edgecolor='red', linewidth=3))

plt.title('Graph with Largest Component Enclosed in Red Boundary')
plt.tight_layout()
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


**All** the users in this dataset have their language set as English, which is 40,000 users